fully trained and tested except frechet

In [ ]:
%load_ext autoreload
%autoreload 2

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [ ]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
%cd /content/

Cloning into 'Generative_Models'...
remote: Enumerating objects: 15891, done.
remote: Total 15891 (delta 0), reused 0 (delta 0), pack-reused 15891
Receiving objects: 100% (15891/15891), 733.46 MiB | 40.54 MiB/s, done.
Resolving deltas: 100% (10845/10845), done.
Checking out files: 100% (1697/1697), done.
/content/Generative_Models
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
!pip install colorlog==4.0.2
%cd /content/

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists
/content/Generative_Models
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Connect Drive to save and restore results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists


# Settings

In [ ]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
/


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
#os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
os.chdir('/content/Generative_Models')
print(os.getcwd())

/content/Generative_Models


# only on colab

In [ ]:
!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [ ]:
!tar -xvf data/.pokemon/pokemon.tar --directory data/.pokemon/

Pokemon_LMDB/_training/
Pokemon_LMDB/_training/lock.mdb
Pokemon_LMDB/_training/data.mdb
Pokemon_LMDB/_validation/
Pokemon_LMDB/_validation/lock.mdb
Pokemon_LMDB/_validation/data.mdb
Pokemon_LMDB/meta_info.json


# Dataset loading

In [ ]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [ ]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [ ]:
from transformation.lmdb_transformer import LmdbTransformer

In [ ]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

/content/Generative_Models/transformation/file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [ ]:
import numpy as np
_instance_scale=1.0
for data in val_generator:
    print(np.max(data['images']))
    break
    #print(data.keys())

1.0


In [ ]:
data['images'][0].shape

TensorShape([100, 100, 3])

In [ ]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [ ]:
  menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
              tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(units=latents_dim)]

  venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
              tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(units=latents_dim)]

  dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
              tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
              tf.keras.layers.Dense(units=flat_outputs_shape),
              tf.keras.layers.Reshape(inputs_shape)]

In [ ]:
inputs_shape

(100, 100, 3)

# Model definition

In [ ]:
model_name = dataset_name+'VAE_Dense_reconst_ellwlb'
experiments_dir='/content/drive/My Drive/Results'+sep_local+model_name
print(experiments_dir)

/content/drive/My Drive/Results/pokemonVAE_Dense_reconst_ellwlb


In [ ]:
from training.autoencoding_basic.autoencoders.VAE import VAE as AE

In [ ]:
#inputs_shape=image_size

In [ ]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist

In [ ]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [ ]:
create_if_not_exist(_restore)
_restore

'/content/drive/My Drive/Results/pokemonVAE_Dense_reconst_ellwlb/var_save_dir'

In [ ]:
#to restore trained model, set filepath=_restore

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath= _restore
    )

  DEBUG    | Restore old models ...
  DEBUG    | Restore old models ...
  ERROR    | Faild to restore old models !
  ERROR    | Faild to restore old models !



SavedModel file does not exist at: /content/drive/My Drive/Results/pokemonVAE_Dense_reconst_ellwlb/var_save_dir/pokemonVAE_Dense_reconst_ellwlb_inference_mean.hdf5/{saved_model.pbtxt|saved_model.pb}




  WARNING  | None
  WARNING  | None
  WARNING  | None
  WARNING  | None


Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 25)      100       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 25)      650       
_________________________________________________________________
flatten (Flatten)            (None, 250000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               25000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)            

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 30000)             1530000   
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100, 100, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 100, 3)       0

In [ ]:
ae.compile(metrics=None)

Model: "pokemonVAE_Dense_reconst_ellwlb"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Function (None, 100)          25001250    inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 100)]        0           inference_logvariance[0][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 100)]        0           tf_op_layer_Mul[0][0]            
____________________________________________________________________

# Callbacks

In [ ]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'/content/drive/My Drive/Results/pokemonVAE_Dense_reconst_ellwlb/csv_dir/pokemonVAE_Dense_reconst_ellwlb.csv'

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [ ]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [ ]:
import numpy as np

# Model Training

In [ ]:
#ae.fit(
 #   x=train_ds,
  #  input_kw=None,
   # steps_per_epoch=5,#int(1e4),
    #epochs=5,#int(1e6), 
    #verbose=1,#2,
    #callbacks=[ es, ms, csv_log, sg],
    #workers=-1,
    #use_multiprocessing=True,
    #validation_data=test_ds,
    #validation_steps=5,#int(1e4)
#)
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=train_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000

Epoch 00001: loss improved from inf to 7489.77539, saving model to /content/drive/My Drive/Results/VAE_Dense_reconst_ellwlb/var_save_dir
5000/5000 - 194s - loss: 7489.7754 - x_logits_loss: 7488.4155 - x_logpdf_loss: -1.3493e+00 - z_latents_loss: 2.5256 - val_loss: 3515.9463 - val_x_logits_loss: 3514.4741 - val_x_logpdf_loss: -1.3136e+00 - val_z_latents_loss: 2.5190
Epoch 2/1000000

Epoch 00002: loss improved from 7489.77539 to 6409.88525, saving model to /content/drive/My Drive/Results/VAE_Dense_reconst_ellwlb/var_save_dir
5000/5000 - 195s - loss: 6409.8853 - x_logits_loss: 6408.2886 - x_logpdf_loss: -1.3178e+00 - z_latents_loss: 2.5752 - val_loss: 3170.8752 - val_x_logits_loss: 3169.2483 - val_x_logpdf_loss: -1.3431e+00 - val_z_latents_loss: 2.6165
Epoch 3/1000000

Epoch 00003: loss improved from 6409.88525 to 6268.38770, saving model to /content/drive/My Drive/Results/VAE_Dense_reconst_ellwlb/var_save_dir
5000/5000 - 197s - loss: 6268.3877 - x_logits_loss: 6266.6514 

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=300)
print(f'inception_score mean: {is_mean.compute()}, sigma: {is_sigma.compute()}')

87916544/87910968 [==============================] - 1s 0us/step
calculating the inception_score mean ...


300it [2:21:06, 29.31s/it]

calculating the inception_score sigma ...


/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
300it [2:21:43, 27.38s/it]

inception_score mean: [2.39646835e+36], sigma: 9.28545604468333e+71



## Frechet_inception_distance




In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, train_generator.as_numpy_iterator(), 
                                       tolerance_threshold=1e-6, 
                                       max_iteration=300, 
                                       batch_size=32)


In [ ]:
print(f'frechet inception distance: {fis_score.compute()}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, train_generator.as_numpy_iterator(), 
                                              tolerance_threshold=1e-6, 
                                              max_iteration=300, 
                                              batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

58892288/58889256 [==============================] - 0s 0us/step


300it [05:02,  1.00s/it]

perceptual path length score: 5244.62646484375


## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae,  train_generator.as_numpy_iterator(), 
                                   tolerance_threshold=1e-6, 
                                   max_iteration=300)
print(f'precision score: {_precision_score}')

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00244379s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00180149s
precision score: [0.]


## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, train_generator.as_numpy_iterator(), tolerance_threshold=1e-6, max_iteration=300)
print(f'recall score: {_recall_score}')

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00231266s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.0021708s
recall score: [0.]


# Image Generation

## image reconstruction

### Training dataset

In [ ]:
from training.generators.from_images.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:00, 58.67it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 36.64it/s]


## with Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:00, 38.16it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 51.84it/s]


### Complete Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

32it [00:00, 84.50it/s]


In [ ]:
from training.generators.from_images.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

3it [00:00, 24.97it/s]

10it [00:00, 22.53it/s]
3it [00:00, 23.91it/s]

10it [00:00, 22.84it/s]
3it [00:00, 24.89it/s]

10it [00:00, 24.25it/s]
3it [00:00, 24.41it/s]

10it [00:00, 21.50it/s]
100%|██████████| 4/4 [00:01<00:00,  2.14it/s]
10it [00:00, 23.83it/s]
